<a href="https://colab.research.google.com/github/nelacopot/blockchain/blob/master/PIPR_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
%%writefile cpu.cpp
#include <iostream>
#include <vector>
#include <random>
#include <algorithm>
#include <chrono>
#include <cstdint>
#include <cstdlib>
#include <cmath>

static inline int compute_PSL(const std::vector<int8_t> &seq) {
    const int L = (int)seq.size();
    int max_val = 0;

    for (int k = 1; k < L; k++) {
        int sum = 0;
        // C(k) = sum_{i=0}^{L-k-1} s[i]*s[i+k]
        for (int i = 0; i < L - k; i++) {
            sum += (int)seq[i] * (int)seq[i + k];
        }
        int a = std::abs(sum);
        if (a > max_val) max_val = a;
    }
    return max_val;
}

static inline void random_bipolar(std::vector<int8_t> &seq, std::mt19937 &rng) {
    // seq values in {+1,-1}
    std::uniform_int_distribution<int> dist(0, 1);
    for (auto &x : seq) {
        x = dist(rng) ? (int8_t)-1 : (int8_t)+1;
    }
}

int main(int argc, char **argv) {
    int L = 512;
    long long nfesLmt = 20000;
    unsigned int seed = 0xE24;

    // args: L512 -nfesLmt20000 -seed3620
    for (int i = 1; i < argc; i++) {
        std::string arg = argv[i];
        if (arg.rfind("L", 0) == 0) {
            L = std::stoi(arg.substr(1));
        } else if (arg.rfind("-nfesLmt", 0) == 0) {
            nfesLmt = std::stoll(arg.substr(8));
        } else if (arg.rfind("-seed", 0) == 0) {
            seed = (unsigned int)std::stoul(arg.substr(5));
        }
    }

    std::mt19937 rng(seed);
    std::vector<int8_t> seq(L);

    int best = 1e9;

    auto t0 = std::chrono::high_resolution_clock::now();

    for (long long n = 0; n < nfesLmt; n++) {
        random_bipolar(seq, rng);
        int psl = compute_PSL(seq);
        if (psl < best) best = psl;
    }

    auto t1 = std::chrono::high_resolution_clock::now();
    double sec = std::chrono::duration<double>(t1 - t0).count();

    std::cout << "CPU PSL-only search\n";
    std::cout << "L: " << L << "\n";
    std::cout << "nfesLmt: " << nfesLmt << "\n";
    std::cout << "seed: " << seed << "\n";
    std::cout << "runtime: " << sec << " s\n";
    std::cout << "speed: " << (double)nfesLmt / sec << " eval/s\n";
    std::cout << "best PSL: " << best << "\n";
    return 0;
}


Overwriting cpu.cpp


In [52]:
!g++ -O3 -march=native cpu.cpp -o cpu
!./cpu L5000 -nfesLmt100000 -seed3620

CPU PSL-only search
L: 5000
nfesLmt: 100000
seed: 3620
runtime: 178.824 s
speed: 559.208 eval/s
best PSL: 179


In [25]:
%%writefile gpu.cu
#include <cstdio>
#include <cstdlib>
#include <cstdint>
#include <vector>
#include <algorithm>
#include <chrono>
#include <random>
#include <cuda.h>

#define CUDA_CHECK(call) do {                                 \
  cudaError_t err = call;                                      \
  if (err != cudaSuccess) {                                    \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                  \
            __FILE__, __LINE__, cudaGetErrorString(err));      \
    exit(1);                                                   \
  }                                                            \
} while(0)

// One block per sequence. For each k, threads sum partial correlation and reduce.
// Then thread0 keeps running max over k.
__global__ void psl_direct_kernel(const int8_t* seqs, int32_t* psl, int L, int N) {
  int seq_id = blockIdx.x;
  if (seq_id >= N) return;

  const int8_t* s = seqs + (size_t)seq_id * (size_t)L;

  extern __shared__ int sh[]; // blockDim.x ints

  int best = 0;

  for (int k = 1; k < L; k++) {
    int partial = 0;
    for (int i = threadIdx.x; i < (L - k); i += blockDim.x) {
      partial += (int)s[i] * (int)s[i + k];
    }

    sh[threadIdx.x] = partial;
    __syncthreads();

    // reduce sum
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
      if (threadIdx.x < stride) sh[threadIdx.x] += sh[threadIdx.x + stride];
      __syncthreads();
    }

    if (threadIdx.x == 0) {
      int v = sh[0];
      if (v < 0) v = -v;
      if (v > best) best = v;
    }
    __syncthreads();
  }

  if (threadIdx.x == 0) psl[seq_id] = best;
}

int main(int argc, char** argv) {
  int L = 512;
  long long nfesLmt = 20000;
  int N = 256;
  unsigned int seed = 0xE24;

  // args: L512 -nfesLmt20000 -N256 -seed3620
  for (int i = 1; i < argc; i++) {
    std::string arg = argv[i];
    if (arg.rfind("L", 0) == 0) L = std::stoi(arg.substr(1));
    else if (arg.rfind("-nfesLmt", 0) == 0) nfesLmt = std::stoll(arg.substr(8));
    else if (arg.rfind("-N", 0) == 0) N = std::stoi(arg.substr(2));
    else if (arg.rfind("-seed", 0) == 0) seed = (unsigned int)std::stoul(arg.substr(5));
  }

  if (L < 2) { printf("L must be >=2\n"); return 1; }
  if (N <= 0) { printf("N must be >0\n"); return 1; }

  const int THREADS = 256;

  // Host buffers (CPU RNG)
  std::vector<int8_t> h_seqs((size_t)N * (size_t)L);
  std::vector<int32_t> h_psl(N);

  std::mt19937 rng(seed);
  std::uniform_int_distribution<int> dist(0, 1);

  // Device buffers
  int8_t* d_seqs = nullptr;
  int32_t* d_psl = nullptr;

  size_t seq_bytes = (size_t)N * (size_t)L * sizeof(int8_t);
  size_t psl_bytes = (size_t)N * sizeof(int32_t);

  CUDA_CHECK(cudaMalloc(&d_seqs, seq_bytes));
  CUDA_CHECK(cudaMalloc(&d_psl, psl_bytes));

  int32_t global_best = 1e9;
  long long done = 0;

  auto t0 = std::chrono::high_resolution_clock::now();

  while (done < nfesLmt) {
    // Fill batch on CPU (bipolar +1/-1)
    for (size_t i = 0; i < (size_t)N * (size_t)L; i++) {
      h_seqs[i] = dist(rng) ? (int8_t)-1 : (int8_t)+1;
    }

    // Copy to GPU
    CUDA_CHECK(cudaMemcpy(d_seqs, h_seqs.data(), seq_bytes, cudaMemcpyHostToDevice));

    // Compute PSL on GPU
    psl_direct_kernel<<<N, THREADS, THREADS * (int)sizeof(int)>>>(d_seqs, d_psl, L, N);
    CUDA_CHECK(cudaGetLastError());

    // Copy PSL back
    CUDA_CHECK(cudaMemcpy(h_psl.data(), d_psl, psl_bytes, cudaMemcpyDeviceToHost));

    // Update best + nfes counter
    for (int i = 0; i < N && done < nfesLmt; i++, done++) {
      if (h_psl[i] < global_best) global_best = h_psl[i];
    }
  }

  CUDA_CHECK(cudaDeviceSynchronize());
  auto t1 = std::chrono::high_resolution_clock::now();
  double sec = std::chrono::duration<double>(t1 - t0).count();

  printf("GPU PSL-only search (CPU RNG)\n");
  printf("L: %d\n", L);
  printf("nfesLmt: %lld\n", nfesLmt);
  printf("batch N: %d\n", N);
  printf("seed: %u\n", seed);
  printf("runtime: %.6f s\n", sec);
  printf("speed: %.2f eval/s\n", (double)nfesLmt / sec);
  printf("best PSL: %d\n", global_best);

  cudaFree(d_seqs);
  cudaFree(d_psl);
  return 0;
}


Writing gpu.cu


In [91]:
!nvcc -O3 -gencode arch=compute_75,code=sm_75 gpu.cu -o gpu
!./gpu L100 -nfesLmt100000 -N32 -seed3620

GPU PSL-only search (CPU RNG)
L: 100
nfesLmt: 100000
batch N: 32
seed: 3620
runtime: 0.606102 s
speed: 164988.76 eval/s
best PSL: 11
